In [ ]:
import requests
from bs4 import BeautifulSoup
import math
import re
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the OpenWeather API key from the environment
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")

# Check if the API key was loaded correctly
if OPENWEATHER_API_KEY is None:
    print("API key not found. Please check your .env file.")
    exit()

# WA tide stations (Department of Transport URLs)
DOT_STATIONS_WA = [
    {"name": "Port Hedland", "slug": "port-hedland", "lat": -20.31, "lon": 118.58},
    {"name": "Fremantle Fishing Boat Harbour", "slug": "fremantle-fishing-boat-harbour", "lat": -32.05, "lon": 115.75},
    {"name": "Broome", "slug": "broome", "lat": -17.96, "lon": 122.23},
    {"name": "Onslow", "slug": "onslow", "lat": -21.64, "lon": 115.11},
]

# ----------------------------------------
# HELPER FUNCTIONS
# ----------------------------------------
def haversine(lat1, lon1, lat2, lon2):
    """Calculate great-circle distance between two coordinates (km)."""
    R = 6371  # Earth radius in km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def find_nearest_station(lat, lon):
    """Return nearest DoT tide station from list."""
    nearest = min(DOT_STATIONS_WA, key=lambda s: haversine(lat, lon, s["lat"], s["lon"]))
    return nearest

def get_coordinates(location):
    """Use OpenWeatherMap Geocoding API to get lat/lon."""
    url = f"http://api.openweathermap.org/geo/1.0/direct?q={location}&limit=1&appid={OPENWEATHER_API_KEY}"
    response = requests.get(url)
    data = response.json()

    if not data:
        raise ValueError(f"Could not find coordinates for {location}")
    return data[0]["lat"], data[0]["lon"]

def get_wind_data(lat, lon):
    """Get wind speed/direction from OpenWeatherMap."""
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={OPENWEATHER_API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()

    wind_speed = data["wind"]["speed"]
    wind_dir = data["wind"].get("deg", "N/A")
    return wind_speed, wind_dir

def get_tide_data_dot(slug):
    """Scrape current tide height from DoT WA tide page."""
    url = f"https://www.transport.wa.gov.au/marine/charts-warnings-current-conditions/coastal-data-charts/tide-data/{slug}"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Failed to fetch DoT page for {slug}")
        return None, None

    soup = BeautifulSoup(response.text, "html.parser")
    text = soup.get_text(" ", strip=True)

    # Look for "Recorded Tide: X.XX Metres" pattern
    match = re.search(r"Recorded Tide[: ]+([0-9.]+)\s*Metres", text, re.IGNORECASE)
    if match:
        height = float(match.group(1))
        return height, "current"
    return None, None

# ----------------------------------------
# MAIN EXECUTION
# ----------------------------------------
if __name__ == "__main__":
    location = input("Enter WA port name (e.g., Port Hedland, Fremantle): ")

    try:
        # Step 1: Get coordinates
        lat, lon = get_coordinates(location)

        # Step 2: Find nearest tide station
        nearest = find_nearest_station(lat, lon)

        # Step 3: Fetch data
        V, wind_dir = get_wind_data(lat, lon)  # <-- Wind speed as V
        H, tide_time = get_tide_data_dot(nearest["slug"])  # <-- Tide height as H

        # Step 4: Display
        print(f"\nLocation: {location}")
        print(f"Coordinates: {lat:.2f}, {lon:.2f}")
        print(f"Nearest DoT Tide Station: {nearest['name']}")

        print("\nWind Data:")
        print(f"  Speed (V): {V} m/s")
        print(f"  Direction: {wind_dir}°")

        if H is not None:
            print("\nTide Data:")
            print(f"  Height (H): {H} m ({tide_time})")
        else:
            print("\nCould not fetch tide data from Department of Transport WA.")

        # Parameters
        Pa = 1.225  # Air density at sea level in kg/m³
        Cd = 1.1    # Coefficient of drag for a large ship
        Aw1 = 15 * 320  # Approximate wind-exposed area in m² for a large ship
        Pw = 1025  # Seawater density in kg/m³
        g = 9.81  # Gravitational acceleration in m/s²
        Aw2 = 15 * 320  # Approximate underwater area in m² for a large ship

        # Calculate forces
        Fw = 0.5 * Pa * Cd * Aw1 * V**2  # Wind force
        Fwave = 0.5 * Pw * g * Aw2 * H**2  # Wave force
        dist = 0.1  # Distance from berth to ship in meters
        kp = 10000  # Optimization parameter for line tension

        # Total force (wind + wave + distance component)
        Ftotal = Fw + Fwave + dist * kp

        # Angles of mooring lines in degrees
        angles = [30, 80, 20, 30]  
        dict_tensions = {}

        # Calculate tension for each line based on angle
        for angle in angles:
            cos_theta = math.cos(math.radians(angle))
            tension_line = Ftotal * cos_theta
            dict_tensions[angle] = tension_line

        # Get user input for line angle and number of lines
        input_angles = int(input("\nEnter angle of mooring lines (30, 80, 20): "))
        input_lines = int(input("\nEnter number of mooring lines (1-3): "))

        tension_selected = dict_tensions.get(input_angles, None)

        # Ensure tension_selected is a valid number
        if tension_selected is None:
            raise ValueError("Invalid angle selection!")

        # Define thresholds for safety levels
        if input_lines == 3:
            thresholds = [5.01, 4.75, 4.50, 4.50]
        elif input_lines == 2:
            thresholds = [2.50, 2.25, 2.00, 2.00]
        elif input_lines == 1:
            thresholds = [1.25, 1.00, 0.80, 0.80]
        else:
            raise ValueError("Invalid number of mooring lines!")

        # Check safety levels based on tension thresholds
        if tension_selected > thresholds[0] * tension_selected:
            status = 'Breakage'
        elif tension_selected > thresholds[1] * tension_selected:
            status = 'Dangerous'
        elif tension_selected > thresholds[2] * tension_selected:
            status = 'Moderate'
        else:
            status = 'Safe'

        # Display the result
        print(f"\nMooring Line Tension: {tension_selected:.2f} N")
        print(f"Status Report: {status}")

    except Exception as e:
        print(f"\nError: {e}")



Location: Port Hedland
Coordinates: -20.31, 118.58
Nearest DoT Tide Station: Port Hedland

Wind Data:
  Speed (V): 7.2 m/s
  Direction: 320°

Tide Data:
  Height (H): 3.39 m (current)

Mooring Line Tension: 240324563.64 N
Status Report: Safe


In [35]:
import numpy as np
import matplotlib.pyplot as plt
import control as ctrl
from scipy.optimize import minimize

In [36]:
tau = 10  # time constant
plant = ctrl.tf([1], [tau, 1])  # G(s) = 1 / (10s + 1)

In [37]:
def pid_cost(params):
    Kp, Ki, Kd = params
    pid = ctrl.tf([Kd, Kp, Ki], [1, 0])
    system = ctrl.feedback(pid * plant)
    time = np.linspace(0, 100, 500)
    t, y = ctrl.step_response(system, time)
    error = 1 - y  # target displacement is 1 m
    return np.sum(error**2)  # minimize total squared error

In [38]:
initial = [100000, 50000, 100000]
bounds = [(0, 500000), (0, 200000), (0, 200000)]

In [39]:
result = minimize(pid_cost, initial, bounds=bounds)
Kp_opt, Ki_opt, Kd_opt = result.x

print(f"Optimized Kp: {Kp_opt:.2f}, Ki: {Ki_opt:.2f}, Kd: {Kd_opt:.2f}")

Optimized Kp: 100000.00, Ki: 50000.00, Kd: 100000.00
